1. Importing packages and libraries.

In [ ]:
import numpy as np
import scipy
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.signal import find_peaks, argrelextrema, peak_widths
from scipy.optimize import curve_fit
from pybaselines import Baseline
from sklearn.metrics import auc
import cv2
from scipy.stats import norm
from lmfit import Parameters, Minimizer
from lmfit.models import LorentzianModel,ConstantModel, GaussianModel

2. Preparing an artificial dataset with known means and distributions.

In [ ]:
x1 = np.arange(133, 150, 0.1) #141
y1 = norm.pdf(x1, 141, 1.5)* 14000

x2 = np.arange(107, 125, 0.1) #116
y2 = norm.pdf(x2, 116, 2)* 14000

x3 = np.arange(60, 74, 0.1) #66
y3 = norm.pdf(x3, 66, 1.5)* 18000

x4 = np.arange(42, 56, 0.1) #50
y4 = norm.pdf(x4, 50, 1.5)* 16000

x5 = np.arange(20, 36, 0.1) #28
y5 = norm.pdf(x5, 28, 2)* 15000

x_fill1 = np.arange(36, 42, 0.1)
y_fill1 = x_fill1*0

x_fill2 = np.arange(74, 107, 0.1)
y_fill2 = x_fill2*0

x_fill3 = np.arange(125, 133, 0.1)
y_fill3 = x_fill3*0

x_fill4 = np.arange(56, 60, 0.1)
y_fill4 = x_fill4*0

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(x1, y1, color = 'red')
plt.plot(x2, y2, color = 'blue')
plt.plot(x3, y3, color = 'gold')
plt.plot(x4, y4, color = 'green')
plt.plot(x5, y5, color = 'orange')
plt.plot(x_fill1, y_fill1, color = 'black')
plt.plot(x_fill2, y_fill2, color = 'black')
plt.plot(x_fill3, y_fill3, color = 'black')
plt.plot(x_fill4, y_fill4, color = 'black')

In [ ]:
x_data = pd.DataFrame(np.arange(20, 150, 0.1), columns=["Size in nt"])

y_list = y5.tolist()+y_fill1.tolist()+y4.tolist()+y_fill4.tolist()+y3.tolist()+y_fill2.tolist()+y2.tolist()+y_fill3.tolist()+y1.tolist()
y_data = pd.DataFrame(y_list, columns=["Intensity Values"])

xy_data = pd.concat([x_data, y_data], axis=1)
xy_data

In [ ]:
plt.figure(figsize=(15,6))
plot = plt.plot(xy_data["Size in nt"], xy_data["Intensity Values"])
ax = plt.gca().invert_xaxis()
plt.xlabel("Size in nt")
plt.ylabel("Intensity Values")
plt.show()

3. Finding the peak values and locations.

In [ ]:
peaks_list = []
peaks, values = find_peaks(xy_data["Intensity Values"], height=1000)
peak_values = xy_data.iloc[peaks, 0]
peak_values = list(peak_values)
peak_data = (peak_values)
peaks_list.append(peak_data)

In [ ]:
plt.figure(figsize=(15,6))
plot = plt.plot(xy_data["Size in nt"], xy_data["Intensity Values"])
plt.scatter(peak_values, values.values(), color='red')
ax = plt.gca().invert_xaxis()
plt.xlabel("Size in nt")
plt.ylabel("Grey Values")
plt.show()

In [ ]:
widths, width_heights, left_ips, right_ips = peak_widths(xy_data["Intensity Values"], peaks, rel_height=1)

In [ ]:
from scipy.interpolate import interp1d

def index_to_xdata(xdata, indices):
    "interpolate the values from signal.peak_widths to xdata"
    ind = np.arange(len(xdata))
    f = interp1d(ind,xdata)
    return f(indices)

widths1 = index_to_xdata(xy_data["Size in nt"], widths)
left_ips1 = index_to_xdata(xy_data["Size in nt"], left_ips)
right_ips1 = index_to_xdata(xy_data["Size in nt"], right_ips)

In [ ]:
plt.figure(figsize=(15,6))
plot = plt.plot(xy_data["Size in nt"], xy_data["Intensity Values"])
plt.scatter(peak_values, values.values(), color='red')
plt.hlines(width_heights, left_ips1, right_ips1, color='r')
ax = plt.gca().invert_xaxis()
plt.xlabel("Size in nt")
plt.ylabel("Grey Values")
plt.show()

4. Gaussian fitting. INDEPENDENT

In [ ]:
left_ips = list(left_ips)
right_ips = list(right_ips)
for i, n in enumerate(left_ips):
    left_ips[i] = int(n)
    
for i, n in enumerate(right_ips):
    right_ips[i] = int(n)


In [ ]:
values_list = list(values.values())
values_list_each=[]

for i in values_list:
    for j in i:
        values_list_each.append(j)

In [ ]:
# Initiating the model for all possible positions with sd with Gaussian

model=ConstantModel()
params=model.make_params()

In [ ]:
# Fitting the data based on the Gaussian model

f1 = GaussianModel(prefix='f1_')
f2 = GaussianModel(prefix='f2_') 
f3 = GaussianModel(prefix='f3_') 


# Making guesses
p1=f1.make_params()
p1['f1_center'].set(peak_data[1],min=left_ips1[1], max=right_ips1[1]) #first parameter is the mean, or the peak location on x axis, then where the peak begins and ends
p1['f1_amplitude'].set(values_list_each[1],min=0) #the height of the peak, the point on the y axis where the mean is located. 
#p1['f1_sigma'].set(sd)

p2=f2.make_params()
p2['f2_center'].set(peak_data[2],min=left_ips1[2], max=right_ips1[2])
p2['f2_amplitude'].set(values_list_each[2],min=0)
#p2['f2_sigma'].set(sd)

p3=f3.make_params()
p3['f3_center'].set(peak_data[3],min=left_ips1[3], max=right_ips1[3])
p3['f3_amplitude'].set(values_list_each[3],min=0)
#p3['f3_sigma'].set(sd)

In [ ]:
# Making a compound model of all possible variations

model1 = model+f1
model2 = model+f2
model3 = model+f3
params.update(p1)
params.update(p2)
params.update(p3)

In [ ]:
model1_y = xy_data["Intensity Values"]
model1_y = model1_y[left_ips[1]:right_ips[1]]

model1_x = xy_data["Size in nt"]
model1_x = model1_x[left_ips[1]:right_ips[1]]

model2_y = xy_data["Intensity Values"]
model2_y = model2_y[left_ips[2]:right_ips[2]]

model2_x = xy_data["Size in nt"]
model2_x = model2_x[left_ips[2]:right_ips[2]]

model3_y = xy_data["Intensity Values"]
model3_y = model3_y[left_ips[3]:right_ips[3]]

model3_x = xy_data["Size in nt"]
model3_x = model3_x[left_ips[3]:right_ips[3]]

model_all_x = xy_data["Size in nt"]
model_all_x = model_all_x[left_ips[1]:right_ips[3]]

model_all_y = xy_data["Size in nt"]
model_all_y = model_all_y[left_ips[1]:right_ips[3]]

In [ ]:
xy_data["Size in nt"], xy_data["Intensity Values"]
result1 = model1.fit(data=model1_y, params = params, x = model1_x ) #xy_data["Intensity Values"], xy_data["Size in nt"]
comps1 = result1.eval_components()

result2 = model2.fit(data=model2_y, params = params, x = model2_x ) #xy_data["Intensity Values"], xy_data["Size in nt"]
comps2 = result2.eval_components()

result3 = model3.fit(data=model3_y, params = params, x = model3_x ) #xy_data["Intensity Values"], xy_data["Size in nt"]
comps3 = result3.eval_components()

plt.figure(figsize=(15, 8))
plt.plot(xy_data["Size in nt"], xy_data["Intensity Values"],  label='Initial data', linewidth = 2.5)

# Plotting each peak (component)
for name, comp in comps1.items():
    if name == "constant":
        continue
    else:
        #continue
        plt.plot(model1_x,comp, '--', label=name, linewidth = 2.5) #xy_data["Size in nt"]
        
# Plotting each peak (component)
for name, comp in comps2.items():
    if name == "constant":
        continue
    else:
        #continue
        plt.plot(model2_x,comp, '--', label=name, linewidth = 2.5) #xy_data["Size in nt"]
        
# Plotting each peak (component)
for name, comp in comps3.items():
    if name == "constant":
        continue
    else:
        #continue
        plt.plot(model3_x,comp, '--', label=name, linewidth = 2.5) #xy_data["Size in nt"]                 
        
#ax = plt.gca().invert_xaxis()
plt.xlabel("Nucleotides", fontsize = 15)
plt.ylabel("Intensity Values", fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

In [ ]:
model_all = model+f1+f2+f3

In [ ]:
model_all1 = f1+f2+f3
params.update(p1)
params.update(p2)
params.update(p3)

In [ ]:
# Dependent
xy_data["Size in nt"], xy_data["Intensity Values"]
result = model_all.fit(data=xy_data["Intensity Values"], params = params, x = xy_data["Size in nt"] ) #xy_data["Intensity Values"], xy_data["Size in nt"]
comps = result.eval_components()

plt.figure(figsize=(15, 8))
plt.plot(xy_data["Size in nt"], xy_data["Intensity Values"],  label='Initial data', linewidth = 2.5)

# Plotting each peak (component)
for name, comp in comps.items():
    if name == "constant":
        continue
    else:
        #continue
        plt.plot(xy_data["Size in nt"],comp, '--', label=name, linewidth = 2.5) #xy_data["Size in nt"]                 
        
#ax = plt.gca().invert_xaxis()
plt.xlabel("Nucleotides", fontsize = 15)
plt.ylabel("Intensity Values", fontsize = 15)
plt.legend(fontsize = 15)
plt.show()

2. Selecting intact DNA area.

In [ ]:
x_data = xy_data["Size in nt"]
y_data = xy_data["Intensity Values"]


intact_dna_area = auc(x_data[left_ips[1]:right_ips[1]], y_data[left_ips[1]:right_ips[1]])

In [ ]:
intact_dna_area

3. Selecting cut DNA I area.

In [ ]:
cut_dna_i_area = auc(x_data[left_ips[2]:right_ips[2]], y_data[left_ips[2]:right_ips[2]])

In [ ]:
cut_dna_i_area

4. Selecting cut DNA II area.

In [ ]:
cut_dna_ii_area = auc(x_data[left_ips[3]:right_ips[3]], y_data[left_ips[3]:right_ips[3]])

In [ ]:
cut_dna_ii_area

In [ ]:
plt.figure(figsize=(15,6))
plot = plt.plot(xy_data["Size in nt"], xy_data["Intensity Values"])
plt.fill_between(x_data[left_ips[1]:right_ips[1]], y_data[left_ips[1]:right_ips[1]], color='gold', alpha=0.3)
plt.fill_between(x_data[left_ips[2]:right_ips[2]], y_data[left_ips[2]:right_ips[2]], color='green', alpha=0.3)
plt.fill_between(x_data[left_ips[3]:right_ips[3]], y_data[left_ips[3]:right_ips[3]], color='blue', alpha=0.3)
ax = plt.gca().invert_xaxis()
plt.xlabel("Size in nt")
plt.ylabel("Grey Values")
plt.show()

5. Normalizing the each area with the total area.

In [ ]:
total_area = intact_dna_area+cut_dna_i_area+cut_dna_ii_area
total_area = total_area/2
total_area

In [ ]:
intact_norm_list = []
intact_norm = intact_dna_area/total_area
intact_norm_list.append(intact_norm)
intact_norm_list

In [ ]:
cut_dna_i_norm_list = []
cut_dna_i_norm = cut_dna_i_area/total_area
cut_dna_i_norm_list.append(cut_dna_i_norm)
cut_dna_i_norm_list

In [ ]:
cut_dna_ii_norm_list = []
cut_dna_ii_norm = cut_dna_ii_area/total_area
cut_dna_ii_norm_list.append(cut_dna_ii_norm)
cut_dna_ii_norm_list

In [ ]:
#auc_percent_substrate_list[-1] = 0
#time_points = [0, 10, 30, 60, 180, 360, 1800, 6000]
time_points = [360]
time = pd.DataFrame(time_points, columns=["Time_Points, s"])
auc_int_dna = pd.DataFrame(intact_norm_list, columns=["Intact DNA"])
auc_cut_dna_i = pd.DataFrame(cut_dna_i_norm_list, columns=["Cut DNA I"])
auc_cut_dna_ii = pd.DataFrame(cut_dna_ii_norm_list, columns=["Cut DNA II"])
df = pd.concat([time, auc_int_dna, auc_cut_dna_i, auc_cut_dna_ii], axis=1)
df

In [ ]:
df.to_csv('analysis_results.csv', encoding='utf-8', index = False, header = False)